In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import ndcg_score
import math

In [ ]:
# %cd /content/gdrive/MyDrive/Hệ khuyến nghị

/content/gdrive/MyDrive/Hệ khuyến nghị


In [ ]:
# import zipfile
# with zipfile.ZipFile("/content/gdrive/MyDrive/Hệ khuyến nghị/word2vec_vi_syllables_300dims.zip","r") as zip_ref:
#     zip_ref.extractall("/content/gdrive/MyDrive/Hệ khuyến nghị")

In [ ]:
# !wget https://public.vinai.io/word2vec_vi_syllables_300dims.zip

--2023-01-02 16:22:37--  https://public.vinai.io/word2vec_vi_syllables_300dims.zip
Resolving public.vinai.io (public.vinai.io)... 108.159.227.26, 108.159.227.61, 108.159.227.34, ...
Connecting to public.vinai.io (public.vinai.io)|108.159.227.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1422871284 (1.3G) [application/zip]
Saving to: ‘word2vec_vi_syllables_300dims.zip’

word2vec_vi_syllabl 100%[===================>]   1.32G  19.2MB/s    in 72s     

2023-01-02 16:23:51 (18.8 MB/s) - ‘word2vec_vi_syllables_300dims.zip’ saved [1422871284/1422871284]



In [ ]:
# from gensim.models import KeyedVectors
# from gensim.test.utils import datapath
# wv = KeyedVectors.load_word2vec_format(datapath('/content/gdrive/MyDrive/Hệ khuyến nghị/word2vec_vi_syllables_300dims.txt'), binary=False)

In [ ]:
# Read data
shopee = pd.read_csv('/content/gdrive/MyDrive/Hệ khuyến nghị/kem_info.csv')
shopee=shopee.drop(labels='Unnamed: 0',axis=1)
shopee=shopee.rename(columns = {'name':'product_name'})

In [ ]:
shopee

,product_id,product_name,brand,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",No,Null,169.000 - ₫399.000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,No,Null,419.000,4.9,"10,9kđã bán"
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,Vaseline,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158.000,4.9,"101,5kđã bán"
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,FOCALLURE,Thời gian giao hàng dự kiến cho sản phẩm này l...,56.000,4.9,"106,2kđã bán"
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,No,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135.000,5.0,50kđã bán
...,...,...,...,...,...,...,...
295,12461191193,Kem chống nắng Omi Sun Bears Strong Super Plus...,No,Null,59.000,4.9,323đã bán
296,14991138669,Kem Chống Nắng K Lady Care Sun Cream 50ml,No,Null,248.000,4.8,11đã bán
297,9048731252,Kem Chống Nắng Cell Fusion C,No,Null,150.000 - ₫380.000,5.0,311đã bán
298,15010825241,Kem Chống Nắng Fixderma Shadow 75g,No,Null,202.000 - ₫241.000,5.0,204đã bán


In [ ]:
shopee[shopee['description']=='Null']

,product_id,product_name,brand,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",No,Null,169.000 - ₫399.000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,No,Null,419.000,4.9,"10,9kđã bán"
11,748923319,(Sale sốc) Kem chống nắng omi Sun Bear Plus SP...,No,Null,75.000,5.0,"2,7kđã bán"
12,9751030861,Kem Chống nắng SJM Medical Anti UV Suncream 60g,No,Null,Null,Null,Null
14,11621381325,Kem Chống Nắng SJM Chống Nước Nâng Tone Da Dàn...,No,Null,Null,Null,Null
...,...,...,...,...,...,...,...
295,12461191193,Kem chống nắng Omi Sun Bears Strong Super Plus...,No,Null,59.000,4.9,323đã bán
296,14991138669,Kem Chống Nắng K Lady Care Sun Cream 50ml,No,Null,248.000,4.8,11đã bán
297,9048731252,Kem Chống Nắng Cell Fusion C,No,Null,150.000 - ₫380.000,5.0,311đã bán
298,15010825241,Kem Chống Nắng Fixderma Shadow 75g,No,Null,202.000 - ₫241.000,5.0,204đã bán


## Xử lí dữ liệu, loại bỏ null

In [ ]:
shopee=shopee[shopee['avg_rating']!="Null"]

In [ ]:
shopee['avg_rating'] = shopee['avg_rating'].astype(float)
shopee=shopee[shopee['avg_rating']>=4.0]

<ipython-input-6-cd918a6be54b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shopee['avg_rating'] = shopee['avg_rating'].astype(float)


In [ ]:
shopee.dtypes

product_id        int64
product_name     object
brand            object
description      object
price            object
avg_rating      float64
luot_mua         object
dtype: object

In [ ]:
shopee['price'].isnull().sum()
shopee['price'].fillna('-1', inplace=True)

In [ ]:
price_list = []
shopee['price'].fillna('-1', inplace=True)
for i in shopee['price'].str.split(" - "):
  
  if len(i)==1:
    price_list.append(int(''.join(i[0].split('.'))))
  elif len(i)==2:
    price_list.append(int(''.join(i[1][1:].split('.'))))
  
shopee['price'] = price_list

In [ ]:
# shopee=shopee.drop(labels='luot_mua',axis=1)
# shopee=shopee.drop(labels='price',axis=1)
# shopee=shopee.drop(labels='avg_rating',axis=1)
shopee=shopee.drop(labels='brand',axis=1)


In [ ]:
shopee.head(2)

,product_id,product_name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,399000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419000,4.9,"10,9kđã bán"


In [ ]:
shopee.drop_duplicates(subset=['product_name'])

,product_id,product_name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,399000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419000,4.9,"10,9kđã bán"
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158000,4.9,"101,5kđã bán"
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,Thời gian giao hàng dự kiến cho sản phẩm này l...,56000,4.9,"106,2kđã bán"
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135000,5.0,50kđã bán
...,...,...,...,...,...,...
295,12461191193,Kem chống nắng Omi Sun Bears Strong Super Plus...,Null,59000,4.9,323đã bán
296,14991138669,Kem Chống Nắng K Lady Care Sun Cream 50ml,Null,248000,4.8,11đã bán
297,9048731252,Kem Chống Nắng Cell Fusion C,Null,380000,5.0,311đã bán
298,15010825241,Kem Chống Nắng Fixderma Shadow 75g,Null,241000,5.0,204đã bán


In [ ]:
shopee['luot_mua'] = shopee['luot_mua'].apply(lambda x: x[0:-6])
def process_luotmua(row):
  if 'k' in row and ',' in row:
    return int(row[0:-3])*1000+int(row[-2])*100
  elif 'k' in row:
    return int(row[0:-1])*1000
  else:
    return int(row)
shopee['luot_mua'] = shopee['luot_mua'].apply(process_luotmua)
shopee.sort_values(by=['luot_mua'],ascending=False)

,product_id,product_name,description,price,avg_rating,luot_mua
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,Thời gian giao hàng dự kiến cho sản phẩm này l...,56000,4.9,106200
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158000,4.9,101500
63,2258622483,(Đủ Bill) Kem chống nắng Cell Fusion C Laser /...,INBOX CHAT VỚI SHOP ĐỂ XEM HÌNH SP VÀ BILL MUA...,305000,5.0,54500
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135000,5.0,50000
34,18539937099,Kem chống nắng lâu trôi làm sáng da innisfree ...,ALT Beauty\nThông Tin Sản Phẩm\nFREESHIP-Kem c...,129000,5.0,50000
...,...,...,...,...,...,...
206,13990420779,Kem chống nắng vật lý nâng tone Peacholic - Hà...,Kem chống nắng vật lý nâng tone Peacholic - Hà...,155000,5.0,5
235,17545615416,Kem chống nắng AMPLE:N Hyaluronshot Suncare SP...,Shop cam kết tất cả các sản phẩm của shop đều ...,184000,5.0,4
205,17525751684,Kem chống nắng chính hãng lâu trôi làm trắng d...,Kem chống nắng chính hãng lâu trôi Innisfree S...,199000,5.0,4
228,18845483508,Kem Chống Nắng Shiseido Spf50 + + + 100ml Bảo ...,Null,204000,5.0,3


In [ ]:
df=shopee

## TF-IDF

In [ ]:
df=shopee[['product_id', 'product_name', 'description', 'luot_mua']]
# df.drop_duplicates(inplace=True)
df=df[df['description']!='Null']


In [ ]:
index_dup = df['product_name'].drop_duplicates().index.tolist()

In [ ]:
df = df.loc[index_dup, :]
df.reset_index(drop=True, inplace=True)

In [ ]:
df['luot_mua'] = df['luot_mua']/df['luot_mua'].max()

In [ ]:
df['luot_mua']= df['luot_mua']/df['luot_mua'].max()

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/Hệ khuyến nghị/itembasecsv')

In [ ]:
# Break up the big genre string into a string array
df['description'] = df['description'].str.split('|')
# Convert genres to string value
df['description'] = df['description'].fillna("").astype('str')

In [ ]:
def preprocessing(text):
  text = text.lower()

  html = re.compile("<.*?>")
  text = html.sub(r"", text)

  text = re.sub("[^\w\s]", "", text)
  text = re.sub('\n','',text)

  return text

In [ ]:
process_discription = []
# Recommend dựa trên description
for i in df['description'].values:
  process_discription.append(preprocessing(i))

In [ ]:
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(process_discription)
tfidf_matrix

<207x5648 sparse matrix of type '<class 'numpy.float64'>'
	with 41017 stored elements in Compressed Sparse Row format>

In [ ]:

cosine_sim = linear_kernel(tfidf_matrix.toarray(), tfidf_matrix.toarray())

In [ ]:
df['product_id']

0       5386949803
1       3240946968
2      21239933571
3      11506865254
4      18558394870
          ...     
202    19824983620
203    12125389908
204    19047195116
205      165944581
206     4586385199
Name: product_id, Length: 207, dtype: int64

In [ ]:
df

,product_id,product_name,description,luot_mua
0,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"['""Vaseline tin rằng bất cứ ai cũng xứng đáng ...",0.955744
1,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,['Thời gian giao hàng dự kiến cho sản phẩm này...,1.000000
2,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,['ALT Beauty\nLưu ý: sản phẩm được sản xuất tạ...,0.470810
3,11506865254,(MẪU MỚI) Kem Chống Nắng Nâng Tone da Innisfre...,[' Kem Chống Nắng Nâng Tone da Innisfree Tone ...,0.035782
4,18558394870,Kem Chống Nắng PrettySkin - Pretty Skin Sun Cr...,['Kem Chống Nắng Pretty Skin Sun Cream SPF 50+...,0.000443
...,...,...,...,...
202,19824983620,Kem chống nắng nâng tone ma:nyo Foundation-Fre...,['✅HƯỚNG DẪN SỬ DỤNG✅ Trong bước cuối cùng của...,0.011299
203,12125389908,[MẪU MỚI 2022] Kem chống nắng cho da dầu EUCER...,['Amélie _ Dược mỹ phẩm Pháp 🇫🇷\nAmélie cam kế...,0.000471
204,19047195116,Kem Chống Nắng La Roche Posay dành cho da nhạy...,['✅ Vie Beauté Đại Lý Phân Phối Chính Thức Dượ...,0.000254
205,165944581,KEM CHỐNG NẮNG INNISFREE DAILY UV PROTECTION C...,['KEM CHỐNG NẮNG INNISFREE DAILY \nUV PROTECTI...,0.015066


## Content base

In [ ]:
df=shopee[['product_id', 'product_name', 'description', 'luot_mua']]
# df.drop_duplicates(inplace=True)
df=df[df['description']!='Null']


In [ ]:
index_dup = df['product_name'].drop_duplicates().index.tolist()

In [ ]:
df = df.loc[index_dup, :]
df.reset_index(drop=True, inplace=True)

In [ ]:
df['luot_mua'] = df['luot_mua']/df['luot_mua'].max()

In [ ]:
df['luot_mua']= df['luot_mua']/df['luot_mua'].max()

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/Hệ khuyến nghị/itembasecsv')

In [ ]:
# Break up the big genre string into a string array
df['description'] = df['description'].str.split('|')
# Convert genres to string value
df['description'] = df['description'].fillna("").astype('str')

In [ ]:
def preprocessing(text):
  text = text.lower()

  html = re.compile("<.*?>")
  text = html.sub(r"", text)

  text = re.sub("[^\w\s]", "", text)
  text = re.sub('\n','',text)
  # # remove English stopwords
  # text = text.split(" ")
  # stops = set(stopwords.words("english"))
  # text = [w for w in text if not w.lower() in stops]
  # text = " ".join([i for i in text])
  # text = ' '.join(text)

  return text

In [ ]:
process_discription = []
# Recommend dựa trên description
for i in df['description'].values:
  process_discription.append(preprocessing(i))

In [ ]:
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(process_discription)
tfidf_matrix

<207x5648 sparse matrix of type '<class 'numpy.float64'>'
	with 41017 stored elements in Compressed Sparse Row format>

In [ ]:

cosine_sim = linear_kernel(tfidf_matrix.toarray(), tfidf_matrix.toarray())

In [ ]:
df['product_id']

0       5386949803
1       3240946968
2      21239933571
3      11506865254
4      18558394870
          ...     
202    19824983620
203    12125389908
204    19047195116
205      165944581
206     4586385199
Name: product_id, Length: 207, dtype: int64

In [ ]:
df

,product_id,product_name,description,luot_mua
0,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"['""Vaseline tin rằng bất cứ ai cũng xứng đáng ...",0.955744
1,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,['Thời gian giao hàng dự kiến cho sản phẩm này...,1.000000
2,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,['ALT Beauty\nLưu ý: sản phẩm được sản xuất tạ...,0.470810
3,11506865254,(MẪU MỚI) Kem Chống Nắng Nâng Tone da Innisfre...,[' Kem Chống Nắng Nâng Tone da Innisfree Tone ...,0.035782
4,18558394870,Kem Chống Nắng PrettySkin - Pretty Skin Sun Cr...,['Kem Chống Nắng Pretty Skin Sun Cream SPF 50+...,0.000443
...,...,...,...,...
202,19824983620,Kem chống nắng nâng tone ma:nyo Foundation-Fre...,['✅HƯỚNG DẪN SỬ DỤNG✅ Trong bước cuối cùng của...,0.011299
203,12125389908,[MẪU MỚI 2022] Kem chống nắng cho da dầu EUCER...,['Amélie _ Dược mỹ phẩm Pháp 🇫🇷\nAmélie cam kế...,0.000471
204,19047195116,Kem Chống Nắng La Roche Posay dành cho da nhạy...,['✅ Vie Beauté Đại Lý Phân Phối Chính Thức Dượ...,0.000254
205,165944581,KEM CHỐNG NẮNG INNISFREE DAILY UV PROTECTION C...,['KEM CHỐNG NẮNG INNISFREE DAILY \nUV PROTECTI...,0.015066


In [ ]:
# Build a 1-dimensional array with movie titles
name = df['product_name']
indices = pd.Series(df.index, index=df['product_name'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def recommendations(product_name, a_weight = 0.2, b_weight = 0.8):
    idx = indices[product_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:101]
    df_indices = [i[0] for i in sim_scores]
    
    luot_mua = df.loc[df_indices, 'luot_mua']
    temp = pd.DataFrame(sim_scores, columns = ['index', 'score'])
    temp.index=temp['index']
    temp.drop(['index'], axis=1, inplace=True)
    score = temp['score']*a_weight+luot_mua*b_weight
    score = score.sort_values(ascending=False)
    df_indices = score.index.values.tolist()

    return name.iloc[df_indices].values[:20]

In [ ]:
recommendations('Kem chống nắng La roche Posay Anthelios SPF 50+, kem chống nắng cho da dầu - Thi Vũ')

array(['Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tông Che Lỗ Chân Lông/Làm Mềm Mịn Da 25g',
       'Serum chống nắng cơ thể Vaseline 50x bảo vệ da với SPF 50+ PA++++ giúp da sáng hơn gấp 2X 300ml',
       '(Đủ Bill) Kem chống nắng Cell Fusion C Laser / Clear / Toning Suncreen Đỏ / Xanh',
       'Kem chống nắng lâu trôi làm sáng da innisfree Intensive Triple Care Sunscreen SPF50+ - ALT Beauty',
       'Kem chống nắng kiềm dầu nâng tông Sunplay Skin Aqua Tone Up UV Milk 80ml - ALT Beauty',
       'Kem Chống Nắng La Roche Posay Anthelios Anti-Shine Dry Touch Gel Cream SPF 50+ Cho Da Dầu - Cila House',
       'Kem chống nắng dưỡng da PERFECT DIARY SPF50+ bảo vệ khỏi tia UV PA+++ dưỡng ẩm dung tích 60ml',
       'Kem chống nắng Skin 1004 Madagascar Centella Airfit Suncream Plus SPF50+ PA++++ 50ml',
       'Mẫu mới kem Chống Nắng The SAEM Eco Earth Power Sun Cream EX SPF50+ PA+++ 50ml',
       'Kem Chống Nắng Ngăn Ngừa Tia UVA và UVB The Saem Eco Earth Sun Cream 50g',
       'Kem chống nắn

In [ ]:
recommendations( 'Kem chống nắng Năng động Cancer Council Active SPF 50+/ PA ++++')

array(['Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tông Che Lỗ Chân Lông/Làm Mềm Mịn Da 25g',
       '(Đủ Bill) Kem chống nắng Cell Fusion C Laser / Clear / Toning Suncreen Đỏ / Xanh',
       'Kem chống nắng lâu trôi làm sáng da innisfree Intensive Triple Care Sunscreen SPF50+ - ALT Beauty',
       'Kem chống nắng kiềm dầu nâng tông Sunplay Skin Aqua Tone Up UV Milk 80ml - ALT Beauty',
       '[Hàng chính hãng] - Kem Body Oliadi Dưỡng Trắng Toàn Thân Với Công Nghệ Hạt Nano Phân Tử 200ml',
       'Kem Chống Nắng La Roche Posay Anthelios Anti-Shine Dry Touch Gel Cream SPF 50+ Cho Da Dầu - Cila House',
       'Kem chống nắng dưỡng da PERFECT DIARY SPF50+ bảo vệ khỏi tia UV PA+++ dưỡng ẩm dung tích 60ml',
       'Kem chống nắng Skin 1004 Madagascar Centella Airfit Suncream Plus SPF50+ PA++++ 50ml',
       'Kem chống nắng Dưỡng ẩm Cancer Council Moisturizing SPF 50+/ PA ++++',
       'Mẫu mới kem Chống Nắng The SAEM Eco Earth Power Sun Cream EX SPF50+ PA+++ 50ml',
       'Kem Chống Nắng Ngă

In [ ]:
df_user = pd.read_csv("/content/gdrive/MyDrive/Hệ khuyến nghị/kem_review.csv")
df_user = df_user[df_user['product_name'].isin(df['product_name'].values.tolist())]

In [ ]:
df_user['cmt_date'] = pd.to_datetime(df_user['cmt_date'])

In [ ]:
df_user = df_user.sort_values(['username', 'cmt_date'], ascending=True)

In [ ]:
df_user['username'].value_counts()

t*****1       446
t*****2       438
t*****3       396
t*****9       377
n*****1       370
             ... 
kien392003      1
kien_linh       1
kiencao527      1
kienchu.96      1
zzzuliiii       1
Name: username, Length: 51926, dtype: int64

In [ ]:
test20_user = df_user['username'].value_counts().index[170:190].tolist()
user_dict = {}
for user in test20_user:
  temp = df_user[df_user['username']==user]
  temp.sort_values(by = ['cmt_date'], ascending=True, inplace=True)

  user_dict[user] = temp.iloc[0, 5]

# Với mỗi user lấy ngẫu nhiên 1 sp đã mua

In [ ]:
user_dict

{'p*****g': 'Kem chống nắng Cell Fusion C Laser Sunscreen 100 SPF50+/PA+++',
 'm*****h': 'Kem chống nắng Cell Fusion C Laser Sunscreen 100 SPF50+/PA+++',
 'd*****h': 'Kem chống nắng Neutrogena U.S Dry Touch SPF 50 88ml - 101016744',
 'v*****6': 'Kem chống nắng kiềm dầu Innisfree Tone Up No Sebum Sunscreen 50ml SPF 50+ PA ++++',
 'c*****7': 'Kem Chống Nắng Cell Fusion C Toning Sunscreen 100',
 'h*****.': 'Kem Chống Nắng Some By Mi Truecica Mineral Calming Tone Up 50ml',
 'c*****0': 'Kem chống nắng Neutrogena U.S Dry Touch SPF 50 88ml - 101016744',
 'q*****9': 'Kem chống nắng Cell Fusion C Laser Đỏ / Clear xanh  / Toning Suncreen hồng / 10ml & 50ml',
 'k*****n': 'Kem Chống Nắng Eucerin Sun 50ml',
 'b*****i': '(Đủ Bill) Kem chống nắng Cell Fusion C Laser / Clear / Toning Suncreen Đỏ / Xanh',
 'a*****7': 'Kem Chống Nắng Eucerin Sun 50ml',
 'c*****5': 'Kem chống nắng Skin1004 Madagascar Centella Air-Fit SunCream Plus SPF50+',
 'y*****8': 'Kem chống nắng Elta MD UV SPF 41 85G',
 'b*****e': '

In [ ]:
df_us = pd.DataFrame(columns=['username'])
df_us['username']=test20_user
df_us

,username
0,p*****g
1,m*****h
2,d*****h
3,v*****6
4,c*****7
5,h*****.
6,c*****0
7,q*****9
8,k*****n
9,b*****i


In [ ]:
user_dict

{'p*****g': 'Kem Chống Nắng La Roche Posay Anthelios Anti-Shine Dry Touch Gel Cream SPF 50+ Cho Da Dầu - Cila House',
 'm*****h': 'Kem chống nắng La roche Posay Anthelios SPF 50+, kem chống nắng cho da dầu - Thi Vũ',
 'd*****h': 'Kem chống nắng dưỡng da PERFECT DIARY SPF50+ bảo vệ khỏi tia UV PA+++ dưỡng ẩm dung tích 60ml',
 'v*****6': 'Kem chống nắng dành cho Da mặt hàng ngày Cancer Council Face Day Wear SPF 50+/ PA ++++',
 'c*****7': 'Kem Chống nắng dưỡng da trắng mịn tối ưu SUNPLAY SKIN AQUA CLEAR WHIT (SPF 50+,PA++++)',
 'h*****.': 'Kem chống nắng Avene Cleanance Solaire SPF 50 cho da mụn mẫu mới - Cila House',
 'c*****0': 'Serum chống nắng cơ thể Vaseline 50x bảo vệ da với SPF 50+ PA++++ giúp da sáng hơn gấp 2X 300ml',
 'q*****9': 'Kem chống nắng Skin1004 Madagascar Centella Air-Fit SunCream Plus SPF50+',
 'k*****n': 'Kem Chống Nắng Innisfree Intensive Sunscreen',
 'b*****i': 'Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tông Che Lỗ Chân Lông/Làm Mềm Mịn Da 25g',
 'a*****7': 'Kem ch

In [ ]:
# df_pre=pd.DataFrame(columns = ['user_name', 'product_name','product_pre'])
# predict_dict = {}
# list_user = []
# list_prod = []
# list_pred = []
# for key in user_dict.keys():
#     list_user = list_user + [key for i in range(20)]
#     list_prod = list_prod + [user_dict[key] for i in range(20)]
#     predict = recommendations(user_dict[key], 0.3, 0.7)
#     list_pred = list_pred + predict.tolist()
    

# df_pre['user_name'] = list_user
# df_pre['product_name'] = list_prod
# df_pre['product_pre'] = list_pred
# df_pre

In [ ]:
groundtruth = df_user[df_user['username'].isin(df_us['username'].values)][['username', 'product_name']].drop_duplicates()
groundtruth.index = groundtruth['username']
groundtruth.drop(['username'], axis=1, inplace=True)

In [ ]:
groundtruth.index.value_counts()

k*****n    31
c*****5    29
m*****h    29
m*****u    28
c*****e    28
b*****i    27
c*****7    27
v*****6    27
h*****.    27
b*****e    27
c*****0    26
q*****9    26
y*****8    25
h*****m    25
m*****i    23
n*****m    23
p*****g    23
a*****7    23
m*****o    22
d*****h    22
Name: username, dtype: int64

In [ ]:
groundtruth

,product_name
username,
a*****7,Kem Chống Nắng Eucerin Sun 50ml
a*****7,(Bao bì mới) Kem chống nắng kiềm dầu và nâng t...
a*****7,Kem chống nắng Skin 1004 Madagascar Centella A...
a*****7,(Đủ Bill) Kem chống nắng Cell Fusion C Laser /...
a*****7,Kem chống nắng đa năng Paula’s Choice SuperLig...
...,...
y*****8,[Bạn mới nhập COSHBM -50% đơn 0Đ] Kem chống ti...
y*****8,Kem Chống nắng SJM Medical Anti UV Suncream 60g
y*****8,Kem Chống Nắng La Roche Posay Anthelios Anti-S...


### PtopK

In [ ]:
df_pre=pd.DataFrame(columns = ['user_name', 'product_name','product_pre'])
predict_dict = {}
list_user = []
list_prod = []
list_pred = []
for key in user_dict.keys():
    list_user = list_user + [key for i in range(20)]
    list_prod = list_prod + [user_dict[key] for i in range(20)]
    predict = recommendations(user_dict[key])
    list_pred = list_pred + predict.tolist()
    

df_pre['user_name'] = list_user
df_pre['product_name'] = list_prod
df_pre['product_pre'] = list_pred
df_pre

,user_name,product_name,product_pre
0,p*****g,Kem Chống Nắng La Roche Posay Anthelios Anti-S...,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...
1,p*****g,Kem Chống Nắng La Roche Posay Anthelios Anti-S...,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...
2,p*****g,Kem Chống Nắng La Roche Posay Anthelios Anti-S...,(Đủ Bill) Kem chống nắng Cell Fusion C Laser /...
3,p*****g,Kem Chống Nắng La Roche Posay Anthelios Anti-S...,Kem chống nắng lâu trôi làm sáng da innisfree ...
4,p*****g,Kem Chống Nắng La Roche Posay Anthelios Anti-S...,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...
...,...,...,...
395,m*****i,Kem chống nắng Skin 1004 Madagascar Centella A...,Kem chống nắng Nivea Shine Control SPF 50 UV Face
396,m*****i,Kem chống nắng Skin 1004 Madagascar Centella A...,Kem Chống nắng dưỡng da trắng mịn tối ưu SUNPL...
397,m*****i,Kem chống nắng Skin 1004 Madagascar Centella A...,Kem chống nắng Cell Fusion C Laser Đỏ / Clear ...
398,m*****i,Kem chống nắng Skin 1004 Madagascar Centella A...,Kem chống nắng La roche Posay Anthelios SPF 50...


In [ ]:
def PtopK(username):
  count = 0
  for m in df_pre[df_pre['user_name']==username].product_pre:
    for s in groundtruth[groundtruth.index==username].product_name:
      if(m == s):
        count+=1

  PtopK = count/len(df_pre[df_pre['user_name']==username])
  return PtopK

In [ ]:
sum_ptopk=0
for username in df_us['username']:
  sum_ptopk += PtopK(username)

print("mean PtopK of 20 users is: ", sum_ptopk/len(df_us))


mean PtopK of 20 users is:  0.38749999999999996


### MRR

In [ ]:
def get_pred_user(username, df_pre, groundtruth):
  MRR_predict = []
  pred = df_pre[df_pre['user_name']==username]['product_pre'].values.tolist()
  true = groundtruth.loc[username, 'product_name'].values.tolist()
  for i in pred:
    if i in true:
      MRR_predict.append(1)
    else:
      MRR_predict.append(0)
  return MRR_predict


In [ ]:
def RRu_score(MRR_predict):
    RRu = 0
    for i in range(len(MRR_predict)):
      if MRR_predict[i] == 1:
        rr = 1/(i+1)
        RRu+=rr
        break;
    # print(RRu)
    return RRu

In [ ]:
def MRR(ListUser):
  MRR = 0
  
  for user in ListUser:
      MRR_predict = get_pred_user(user, df_pre, groundtruth)
      RRu = RRu_score(MRR_predict)
      MRR += RRu
      print("MRRu score for user {} : {}".format(user, RRu))
  return (1/len(ListUser))*MRR

In [ ]:
ListUser = df_us['username'].values.tolist()

MRR_score = MRR(ListUser)

MRRu score for user p*****g : 1.0
MRRu score for user m*****h : 0.5
MRRu score for user d*****h : 1.0
MRRu score for user v*****6 : 1.0
MRRu score for user c*****7 : 1.0
MRRu score for user h*****. : 1.0
MRRu score for user c*****0 : 1.0
MRRu score for user q*****9 : 1.0
MRRu score for user k*****n : 1.0
MRRu score for user b*****i : 1.0
MRRu score for user a*****7 : 1.0
MRRu score for user c*****5 : 1.0
MRRu score for user y*****8 : 1.0
MRRu score for user b*****e : 0.5
MRRu score for user m*****o : 1.0
MRRu score for user m*****u : 0.3333333333333333
MRRu score for user c*****e : 0.3333333333333333
MRRu score for user h*****m : 0.5
MRRu score for user n*****m : 0.125
MRRu score for user m*****i : 1.0


In [ ]:
print(MRR_score)

0.8145833333333334


### NDCG score

In [ ]:
def DCG_score(predict_ranking):
    DCG = 0
    for i in range(len(predict_ranking)):
        DCG += predict_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def iDCG_score(true_ranking):
    DCG = 0
    for i in range(len(true_ranking)):
        DCG += true_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def NDCG_for_listUser(ListUser):
    ndcg = 0
    for user in ListUser:
        
        prec_ranking = get_pred_user(user, df_pre, groundtruth)
        true_ranking = sorted(prec_ranking, reverse=True)
        print("predict_ranking")
        print(prec_ranking)
        print("true_ranking")
        print(true_ranking)
        dcg = DCG_score(prec_ranking)
        idcg = iDCG_score(true_ranking)
        try:
          ndcg += dcg/idcg
          print("ndcg for user {}: {}".format(user, dcg/idcg))
        except:
          ndcg +=0
        
    return (1/len(ListUser))*ndcg

In [ ]:
ListUser = df_us['username'].values.tolist()

NDCG_score = NDCG_for_listUser(ListUser)

predict_ranking
[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
true_ranking
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg for user p*****g: 0.8259297484096785
predict_ranking
[0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg for user m*****h: 0.7042864179977938
predict_ranking
[1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg for user d*****h: 0.7815087093360953
predict_ranking
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg for user v*****6: 0.8940884741471546
predict_ranking
[1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg for user c*****7: 0.7827219680916239
predict_ranking
[1, 1, 0, 0, 1, 0, 

In [ ]:
NDCG_score

0.7694822259040799